In [10]:
from collections import defaultdict
import numpy as np
import random
from sklearn.metrics.pairwise import cosine_similarity

# 1. Class Cluster & Member

In [11]:
# Member store info of data point: tf-idf, news group of d, file name of d.

class Member():
    def __init__(self, r_d, label = None, doc_id = None):
        self._r_d = r_d
        self._label = label
        self._doc_id = doc_id

In [12]:
# Cluster store the data points

class Cluster():
    def __init__(self):
    # The underscore prefix is meant as a hint that a variable or method starting with "_" is intended for internal use
        self._centroid = None
        self._members = []
    
    # Reset members (data points) in the list of centroids
    def reset_members(self):
        self._members = []
        
    # Set centroid
    def set_centroid(self, new_centroid):
        self._centroid = new_centroid
    
    def add_member(self, new_member):
        self._members.append(new_member)

# 2. Class K - Means

In [23]:
class Kmeans():
    # Init with num_cluster, create K list store member of K centroid
    def __init__(self, num_clusters):
        self._num_clusters = num_clusters
        self._clusters = []
        for i in range(self._num_clusters):
            self._clusters.append(Cluster())
        self._E = [] # list of centroids
        self._S = 0 # Overall similarity
        
    # Load data
    def load_data(self, path = "C:/Users/HH/OneDrive - Hanoi University of Science and Technology/ML basics/Session1/20news-bydate/"):
        # Get data from each line with label, doc_id, index & tfidf of its vocab
        def sparse_to_dense(sparse_r_d, vocab_size):
            r_d = [0.0 for i in range(vocab_size)]
            
            # Split space & : in context data of each line 
            # Get index (id vocal of each line) & tfidfs
            indices_and_tfidfs = sparse_r_d.split()
            for index_and_tfidf in indices_and_tfidfs:
                index = int(index_and_tfidf.split(':')[0])
                tfidf = float(index_and_tfidf.split(':')[1])
                r_d[index] = tfidf
            return np.array(r_d)
        
        # Open file (newsgroup, id, context)
        with open(path + "data_tf_idf.txt") as file:
            content = file.read().splitlines()
        
        # Get size of words_idf file (number of words)
        with open(path + "words_idf.txt") as file:
            vocab_size = len(file.read().splitlines())
            
        self._data = []
        for line in content:
            data_lines = line.split("<fff>")
            r_d = sparse_to_dense(data_lines[2], vocab_size)
            self._data.append(Member(r_d = r_d, label = int(data_lines[0]), doc_id = int(data_lines[1])))
    
    # Random element to init centroid
    def random_init(self, seed_value):
        random.seed(seed_value)
        # Crawl list members
        members = [member._r_d for member in self._data]
        
        # random.choice: random sample from 1-D array
        # Same as np.arange without same num in array
        pos = np.random.choice(len(self._data), self._num_clusters, replace = False)
        centroid = []
        for i in pos:
            centroid.append(members[i])
        # Update centroid
        self._E = centroid
        for i in range(self._num_clusters):
            self._clusters[i].set_centroid(centroid[i])
    
    # Calculate similarity from centroid to another member
    # https://www.machinelearningplus.com/nlp/cosine-similarity/
    def compute_similarity(self, member, centroid):
        # K(X, Y) = <X, Y> / (||X||*||Y||)
        return cosine_similarity([member._r_d], [centroid])

    # Assign member to cluster
    def select_cluster_for(self, member):
        best_fit_cluster = None
        
        # min(cos) = -1
        max_similarity = -1
        
        for cluster in self._clusters:
            similarity = self.compute_similarity(member, cluster._centroid)
            if similarity > max_similarity:
                best_fit_cluster = cluster
                max_similarity = similarity
        best_fit_cluster.add_member(member)
        return max_similarity
    
    # Recompute centroid
    def update_centroid_of(self, cluster):
        # list of all tf-idf representation
        member_r_d = [member._r_d for member in cluster._members]
        avg_r_d = np.mean(member_r_d, axis = 0)
        sqrt_sum = np.sqrt(np.sum(avg_r_d**2))
        new_centroid = avg_r_d * 1./sqrt_sum
        
        # update centroid
        cluster._centroid = new_centroid
        
    def stopping_condition(self, criterion, threshold):
        criteria = ["centroid", "similarity", "max_iters"]
        assert criterion in criteria
        if criterion == "max_iters":
            if self._iteration >= threshold:
                return True
            else:
                return False
        if criterion == "centroid":
            E_new = [list(cluster._centroid) for cluster in self._clusters]
            E_new_minus_E = [centroid for centroid in E_new if centroid not in self._E]
            if len(E_new_minus_E) <= threshold:
                return True
            else:
                return False
        if criterion == "similarity":
            new_S_minus_S = self._new_S - self._S
            self._S = self._new_S
            if new_S_minus_S <= threshold:
                return True
            else:
                return False
    # Loop update cluster until convergence 
    # Step 1: For each instance, assign it to nearest centroid
    # Step 2: For each cluster, recompute its centroid
    def run(self, seed_value, criterion, threshold):
        self.random_init(seed_value)
        self._iteration = 0
        
        while True:
            # reset clusters, retain only centroids
            for cluster in self._clusters:
                cluster.reset_members()
                self._new_S = 0
            # select nearest member for each cluster
            for member in self._data:
                max_s = self.select_cluster_for(member)
                self._new_S += max_s
            # recompute the centroids
            for cluster in self._clusters:
                self.update_centroid_of(cluster)
            # stop loop by stopping condition
            self._iteration += 1
            if self.stopping_condition(criterion, threshold):
                break
    def compute_purity(self):
        majority_sum = 0
        for cluster in self._clusters:
            member_label = [member._label for member in cluster._members]
            max_label = max([member_label.count(label) for label in range(20)])
            majority_sum += max_label
        return majority* 1./len(self._data)

In [ ]:
kmeans = Kmeans(num_clusters=8)
kmeans.load_data()
kmeans.run(seed_value = 42, criterion='similarity', threshold = 1e-3)
print(kmeans.compute_purity())